In [ ]:
from ultralytics import YOLO
import cv2
from sort import Sort
import numpy as np

In [ ]:
model = YOLO("yolov8n.pt")
tracker = Sort()

In [20]:
cap = cv2.VideoCapture("train.mp4")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Something is wrong with video or camera!")
        break

    results = model.track(frame, persist=True, classes=[0], tracker="bytetrack.yaml", verbose=False, show=False)
    temp_frame = frame.copy()

    for box in results[0].boxes:
        class_id = int(box.cls[0])
        if class_id == 0:
            x1,y1,x2,y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])

        cv2.rectangle(temp_frame, (x1,y1), (x2,y2), (0,255,0), 2)
        label = f"Person {conf:.2f}"
        cv2.putText(temp_frame, label, (x1,y1-10),
                    cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.6, color=(255,255,255), thickness=2)
        
    height, width = frame.shape[:2]
    cv2.line(temp_frame, (0, height // 2), (width, height // 1), (0, 0, 255), 2)
        
    cv2.imshow("track",temp_frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
